# Building a Feature Repository with SageMaker Feature Store
> The aim of this notebook is to demonstrate how to built a central feature repository using Amazon SageMaker Feature Store. Feature Store is used to store, retrieve, and share machine learning features.

- toc: true 
- badges: true
- comments: true
- categories: [aws, ml, sagemaker]
- keyword: [aws, ml, sagemaker, feature store]
- image: images/copied_from_nb/images/2022-08-05-sagemaker-feature-store.jpeg

![](images/2022-08-05-sagemaker-feature-store.jpeg)

# Introduction

Amazon SageMaker Feature Store is a fully managed, purpose-built repository to store, share, and manage features for machine learning (ML) models. Features are inputs to ML models used during training and inference. For example, in an application that recommends a music playlist, features could include song ratings, listening duration, and listener demographics. Features are used repeatedly by multiple teams and feature quality is critical to ensure a highly accurate model. Also, when features used to train models offline in batch are made available for real-time inference, it’s hard to keep the two feature stores synchronized. SageMaker Feature Store provides a secured and unified store for feature use across the ML lifecycle.

![feature-store.png](images/2022-08-05-sagemaker-feature-store/feature-store.png)


# Environment
This notebook is prepared with AWS SageMaker notebook running on `ml.t3.medium` instance and "conda_python3" kernel.

In [1]:
!aws --version

aws-cli/1.22.97 Python/3.8.12 Linux/5.10.102-99.473.amzn2.x86_64 botocore/1.24.19


In [2]:
!cat /etc/os-release

NAME="Amazon Linux"
VERSION="2"
ID="amzn"
ID_LIKE="centos rhel fedora"
VERSION_ID="2"
PRETTY_NAME="Amazon Linux 2"
ANSI_COLOR="0;33"
CPE_NAME="cpe:2.3:o:amazon:amazon_linux:2"
HOME_URL="https://amazonlinux.com/"


In [3]:
!python3 --version

Python 3.8.12


In [4]:
#collapse-output
!conda env list

# conda environments:
#
base                     /home/ec2-user/anaconda3
JupyterSystemEnv         /home/ec2-user/anaconda3/envs/JupyterSystemEnv
R                        /home/ec2-user/anaconda3/envs/R
amazonei_mxnet_p36       /home/ec2-user/anaconda3/envs/amazonei_mxnet_p36
amazonei_pytorch_latest_p37     /home/ec2-user/anaconda3/envs/amazonei_pytorch_latest_p37
amazonei_tensorflow2_p36     /home/ec2-user/anaconda3/envs/amazonei_tensorflow2_p36
mxnet_p37                /home/ec2-user/anaconda3/envs/mxnet_p37
python3               *  /home/ec2-user/anaconda3/envs/python3
pytorch_p38              /home/ec2-user/anaconda3/envs/pytorch_p38
tensorflow2_p38          /home/ec2-user/anaconda3/envs/tensorflow2_p38



# Prepare training and test data
We will use **Iris flower dataset**. It includes three iris species (Iris setosa, Iris virginica, and Iris versicolor) with 50 samples each. Four features were measured for each sample: the length and the width of the sepals and petals, in centimeters. We can train a model to distinguish the species from each other based on the combination of these four features. You can read more about this dataset at [Iris flower data set](https://en.wikipedia.org/wiki/Iris_flower_data_set). The dataset has five columns representing.
1. sepal length in cm
2. sepal width in cm
3. petal length in cm
4. petal width in cm
5. class: Iris Setosa, Iris Versicolour, Iris Virginica